# Regression Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I {**Martha, Mwaura**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Spain Electricity Shortfall Challenge

The government of Spain is considering an expansion of it's renewable energy resource infrastructure investments. As such, they require information on the trends and patterns of the countries renewable sources and fossil fuel energy generation. Your company has been awarded the contract to:

- 1. analyse the supplied data;
- 2. identify potential errors in the data and clean the existing data set;
- 3. determine if additional features can be added to enrich the data set;
- 4. build a model that is capable of forecasting the three hourly demand shortfalls;
- 5. evaluate the accuracy of the best machine learning model;
- 6. determine what features were most important in the model’s prediction decision, and
- 7. explain the inner working of the model to a non-technical audience.

Formally the problem statement was given to you, the senior data scientist, by your manager via email reads as follow:

> In this project you are tasked to model the shortfall between the energy generated by means of fossil fuels and various renewable sources - for the country of Spain. The daily shortfall, which will be referred to as the target variable, will be modelled as a function of various city-specific weather features such as `pressure`, `wind speed`, `humidity`, etc. As with all data science projects, the provided features are rarely adequate predictors of the target variable. As such, you are required to perform feature engineering to ensure that you will be able to accurately model Spain's three hourly shortfalls.
 
On top of this, she has provided you with a starter notebook containing vague explanations of what the main outcomes are. 

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>


For purposes of **data manipulation and analysis**, we will import the following packages and/or libraries:
- `pandas` : It offers data structures and operations for manipulating numerical tables and time series.
- `numpy` : It's the core library for scientific computing in Python.
- `matplotlib` : It is a library for data visualization.
- `seaborn` : It is an alternative library for data visualization.

For purposes of **data preprocessing and model building**, we will import the following packages and/or libraries:
- `sklearn` : It is a library that contains efficient tools for machine learning and statistical modeling.

In [17]:
# Libraries for data loading, data manipulation and data visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for data preparation and model building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Setting global constants to ensure notebook results are reproducible
#PARAMETER_CONSTANT = ###

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

We will load the `df_train` file as a Pandas DataFrame for use in the training and evaluation of the model, and `df_test` to generate our target variable.

In [2]:
# Loading df_train.csv
df = pd.read_csv('Advanced-Regression-Starter-Data/df_train.csv')

# Loading df_test.csv
df_test = pd.read_csv('Advanced-Regression-Starter-Data/df_test.csv')

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

In this section, we will **start by performing basic analysis** which will entail:
- Looking at the sample of the data
- Finding out the number of rows and columns (plus their respective data types) in the dataset
- Viewing summary statistics for numerical columns
- Checking for null values

We will then perform **univariate analysis** of our target variable `load_shortfall_3h` which will entail:
- Looking at its measures of central tendency and other descriptive summary statistics
- Looking at its distribution including its skewness and outliers
- Looking at its relationship with other features

### 3.1 Basic Analysis

Looking at a sample of the dataset:

In [3]:
# Look at a sample from the top of the dataset
df.head()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,...,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.000000,...,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.000000,...,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.000000,...,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667
3,3,2015-01-01 12:00:00,1.000000,level_8,0.0,1.000000,65.333333,56.333333,0.0,1.000000,...,281.895219,283.434104,281.135063,281.135063,283.434104,281.135063,285.394000,281.895219,281.895219,5075.666667
4,4,2015-01-01 15:00:00,1.000000,level_7,0.0,1.000000,59.000000,57.000000,2.0,0.333333,...,280.678437,284.213167,282.252063,282.252063,284.213167,282.252063,285.513719,280.678437,280.678437,6620.666667


In [4]:
# Look at a sample from the bottom of the dataset
df.tail()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,...,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
8758,8758,2017-12-31 09:00:00,1.000000,level_6,0.0,2.666667,89.000000,95.666667,56.666667,4.333333,...,280.816667,281.276667,285.150000,287.573333,280.483333,290.150000,284.816667,279.686667,278.483333,-28.333333
8759,8759,2017-12-31 12:00:00,5.000000,level_6,0.0,2.000000,82.000000,85.000000,26.666667,8.000000,...,283.483333,287.483333,286.483333,288.616667,287.150000,291.150000,287.150000,282.400000,280.150000,2266.666667
8760,8760,2017-12-31 15:00:00,6.333333,level_9,0.4,7.333333,67.666667,71.000000,63.333333,8.333333,...,285.150000,289.816667,283.816667,285.330000,289.150000,286.816667,289.150000,283.956667,281.150000,822.000000
8761,8761,2017-12-31 18:00:00,7.333333,level_8,0.2,7.333333,67.666667,79.000000,63.333333,2.666667,...,283.483333,287.523333,278.816667,281.410000,286.816667,284.150000,289.150000,282.666667,280.816667,-760.000000
8762,8762,2017-12-31 21:00:00,4.333333,level_9,0.0,7.000000,78.666667,68.666667,20.000000,1.666667,...,282.150000,287.483333,276.816667,281.020000,287.150000,285.150000,287.483333,281.396667,280.483333,2780.666667


Since only 20 of the 49 columns have been displayed, we shall use `df.column` to view the names of all columns in order to confirm that the data matches [its description](https://www.kaggle.com/competitions/spain-electricity-shortfall-challenge-2022/data).

In [5]:
# List of columns
df.columns

Index(['Unnamed: 0', 'time', 'Madrid_wind_speed', 'Valencia_wind_deg',
       'Bilbao_rain_1h', 'Valencia_wind_speed', 'Seville_humidity',
       'Madrid_humidity', 'Bilbao_clouds_all', 'Bilbao_wind_speed',
       'Seville_clouds_all', 'Bilbao_wind_deg', 'Barcelona_wind_speed',
       'Barcelona_wind_deg', 'Madrid_clouds_all', 'Seville_wind_speed',
       'Barcelona_rain_1h', 'Seville_pressure', 'Seville_rain_1h',
       'Bilbao_snow_3h', 'Barcelona_pressure', 'Seville_rain_3h',
       'Madrid_rain_1h', 'Barcelona_rain_3h', 'Valencia_snow_3h',
       'Madrid_weather_id', 'Barcelona_weather_id', 'Bilbao_pressure',
       'Seville_weather_id', 'Valencia_pressure', 'Seville_temp_max',
       'Madrid_pressure', 'Valencia_temp_max', 'Valencia_temp',
       'Bilbao_weather_id', 'Seville_temp', 'Valencia_humidity',
       'Valencia_temp_min', 'Barcelona_temp_max', 'Madrid_temp_max',
       'Barcelona_temp', 'Bilbao_temp_min', 'Bilbao_temp',
       'Barcelona_temp_min', 'Bilbao_temp_max', 'Sev

From the list of columns and the sample data above, we confirm that our dataframe has the weather data for the Spain cities: Madrid, Valencia, Bilbao, Seville, and Barcelona.

Next, we will check at the number of records in our dataset.

In [6]:
# Check at the number of records
df.shape

(8763, 49)

Our dataset has a total of **8,763 rows** and **49 columns**. This differs slightly with the given description which totals to 48 columns. The difference is due to the `Unnamed: 0` column which matches the rows index.

The next step is to confirm the datatypes of the respective columns which will guide our feature engineering.

In [7]:
# Confirm column datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            8763 non-null   int64  
 1   time                  8763 non-null   object 
 2   Madrid_wind_speed     8763 non-null   float64
 3   Valencia_wind_deg     8763 non-null   object 
 4   Bilbao_rain_1h        8763 non-null   float64
 5   Valencia_wind_speed   8763 non-null   float64
 6   Seville_humidity      8763 non-null   float64
 7   Madrid_humidity       8763 non-null   float64
 8   Bilbao_clouds_all     8763 non-null   float64
 9   Bilbao_wind_speed     8763 non-null   float64
 10  Seville_clouds_all    8763 non-null   float64
 11  Bilbao_wind_deg       8763 non-null   float64
 12  Barcelona_wind_speed  8763 non-null   float64
 13  Barcelona_wind_deg    8763 non-null   float64
 14  Madrid_clouds_all     8763 non-null   float64
 15  Seville_wind_speed   

All columns are of `float` datatype except columns:
- Unnamed: 0
- time
- Valencia_wind_deg
- Seville_pressure

We are also interested in columns with null values.

In [8]:
# Check for nulls
df.isnull().sum()

Unnamed: 0                 0
time                       0
Madrid_wind_speed          0
Valencia_wind_deg          0
Bilbao_rain_1h             0
Valencia_wind_speed        0
Seville_humidity           0
Madrid_humidity            0
Bilbao_clouds_all          0
Bilbao_wind_speed          0
Seville_clouds_all         0
Bilbao_wind_deg            0
Barcelona_wind_speed       0
Barcelona_wind_deg         0
Madrid_clouds_all          0
Seville_wind_speed         0
Barcelona_rain_1h          0
Seville_pressure           0
Seville_rain_1h            0
Bilbao_snow_3h             0
Barcelona_pressure         0
Seville_rain_3h            0
Madrid_rain_1h             0
Barcelona_rain_3h          0
Valencia_snow_3h           0
Madrid_weather_id          0
Barcelona_weather_id       0
Bilbao_pressure            0
Seville_weather_id         0
Valencia_pressure       2068
Seville_temp_max           0
Madrid_pressure            0
Valencia_temp_max          0
Valencia_temp              0
Bilbao_weather

Only the column `Valencia_pressure` contains null values.

As a last step in the basic analysis, we will generate the summary statistics for the numerical columns.

In [9]:
# look at data statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,8763.0,4381.000000,2529.804538,0.000000,2190.500000,4381.000000,6571.500000,8.762000e+03
Madrid_wind_speed,8763.0,2.425729,1.850371,0.000000,1.000000,2.000000,3.333333,1.300000e+01
Bilbao_rain_1h,8763.0,0.135753,0.374901,0.000000,0.000000,0.000000,0.100000,3.000000e+00
Valencia_wind_speed,8763.0,2.586272,2.411190,0.000000,1.000000,1.666667,3.666667,5.200000e+01
Seville_humidity,8763.0,62.658793,22.621226,8.333333,44.333333,65.666667,82.000000,1.000000e+02
Madrid_humidity,8763.0,57.414717,24.335396,6.333333,36.333333,58.000000,78.666667,1.000000e+02
Bilbao_clouds_all,8763.0,43.469132,32.551044,0.000000,10.000000,45.000000,75.000000,1.000000e+02
Bilbao_wind_speed,8763.0,1.850356,1.695888,0.000000,0.666667,1.000000,2.666667,1.266667e+01
Seville_clouds_all,8763.0,13.714748,24.272482,0.000000,0.000000,0.000000,20.000000,9.733333e+01
Bilbao_wind_deg,8763.0,158.957511,102.056299,0.000000,73.333333,147.000000,234.000000,3.593333e+02


### 3.2 Univariate Analysis

In [ ]:
# plot relevant feature interactions

In [ ]:
# evaluate correlation

In [ ]:
# have a look at feature distributions

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section you are required to: clean the dataset, and possibly create new features - as identified in the EDA phase. |

---
### 4.1 Base Model Data Engineering

For purposes of the creating the base model, we will drop all non-numeric columns and columns with null values.

As outlined in section 3.1, the non-numeric columns are:
- time
- Valencia_wind_deg
- Seville_pressure

The column that contains null values is:
- Valencia_pressure

We will create a `drop_columns` functions that will facilitate this process.

In [9]:
# Function to drop non-numeric columns and columns with null values
def drop_columns(input_df):
    output_df = input_df.copy()
    for column in output_df:
        with_nulls = output_df[column].isna().sum()
        if with_nulls > 0:
            output_df = output_df.drop(column, axis = 1)
        else:
            if column not in df.select_dtypes(include='number').columns:
                output_df = output_df.drop(column, axis = 1)
    return output_df

We will call our function on our training and test datasets.

In [10]:
# The new training dataset
clean_df = drop_columns(df)

In [11]:
# The new test dataset
clean_df_test = drop_columns(df_test)

In [ ]:
# remove missing values/ features

In [ ]:
# create new features

In [ ]:
# engineer existing features

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, you are required to create one or more regression models that are able to accurately predict the thee hour load shortfall. |

---
### 5.1 The Base Model

For the base model, we will use the data as is, that is, without preprocessing.

We will start by splitting the dataset into the response, `y`, and features, `X`.

In [12]:
# Create the target and features
y = clean_df['load_shortfall_3h']
X = clean_df.drop('load_shortfall_3h', axis = 1)

Our target, variable we want to predict, is `load_shortfall_3h`, and the predictors, are all the rest of the columns.

We will then proceed to split our dataset using the `train_test_split` for purposes of training the model we will create, and evaluating its performance.

In [13]:
# Split the train dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 50)

We will use 80% of our dataset for model training purposes, and 20% to validate our model. We have set the random_state value to ensure that the random nature in which rows are picked to be in the validation set is the same each time the split is carried out.

We now proceed in creating our Linear Regression model.

In [14]:
# Declare the model object
lm = LinearRegression()

Our model object has been created. 

We can now fit our model by exposing it to our training data.

In [15]:
# Training the model
lm.fit(X_train, y_train)

LinearRegression()

Having created and trained our model, we proceed to the validation phase, that is, using our 20% reserve data with known target values.

In [16]:
# Make predictions using validation set
validation_preds = lm.predict(X_test)

#### 5.1.1 Evaluate Base Model

We will use the Root Mean Square Error metric to evaluate our model.

In [20]:
#Find the RMSE
rmse = mean_squared_error(y_test, validation_preds, squared = False)

We are aiming for an RMSE value of 3500 or below. Our base model's RMSE is displayed below.

In [21]:
print("Base model RMSE = {}". format(rmse))

Base model RMSE = 4856.6554416172385


#### 5.1.2 Submission From Base Model

We will retrain our model using the combined training and validation datasets, and make predictions for our test set.

In [22]:
# Fit the model using the whole training dataset
lm.fit(X, y)

LinearRegression()

In [23]:
# Make prediction for the target variable
predictions = lm.predict(clean_df_test)

We will store the predictions in a dataframe.

In [26]:
# Store predictions in a dataframe
load_shortfall = pd.DataFrame(predictions, columns = ['load_shortfall_3h'])

# A preview of our dataframe
load_shortfall.head()

,load_shortfall_3h
0,9774.049403
1,9186.230955
2,10194.296086
3,9968.582159
4,9328.980673


The final step is creating the submission csv file in the required format.

In [27]:
# Get Time variable as a dataframe
time = pd.DataFrame({'time': df_test['time']})

# Combine Time DataFrame with the Load_shortfall dataframe
submission = time.join(load_shortfall)

# Write to the result to a csv file
submission.to_csv('submission.csv', index = False)

In [ ]:
# create one or more ML models

In [ ]:
# evaluate one or more ML models

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section you are required to compare the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

In [ ]:
# Compare model performance

In [ ]:
# Choose best model and motivate why it is the best choice

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

In [ ]:
# discuss chosen methods logic